In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# 기본 라이브러리
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter

sns.set()

# 그래프 기본 설정
plt.rcParams['figure.figsize'] = 12, 6
plt.rcParams['font.size'] = 14
plt.rcParams['axes.unicode_minus'] = False
import matplotlib as mpl
import matplotlib.font_manager as fm

# 폰트 파일 경로 지정
# font_path = '/content/drive/MyDrive/NanumFontSetup_TTF_ALL/NanumGothic.ttf'
font_path = '/content/drive/MyDrive/font/NanumGothic.ttf'

# 폰트 등록 및 matplotlib 설정
font_name = fm.FontProperties(fname=font_path).get_name()
fm.fontManager.addfont(font_path)
mpl.rc('font', family=font_name)
# 경고 뜨지 않게
import warnings
warnings.filterwarnings('ignore')

import os

#✅ 데이터셋 불러오기

In [ ]:
notop_multi_df = pd.read_parquet('/content/drive/MyDrive/Colab Notebooks/datathon/주력_비주력상품 데이터/비주력상품/noTop3_multi_buyers_all_events.parquet') # 비주력상품 2회 이상 구매고객 데이터

In [ ]:
df_sampled = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/datathon/주력_비주력상품 데이터/층화샘플링/non_categories_750.csv')

# ✅데이터셋 전처리

# 📦 EDA 통합

## 📈 주력상품 EDA

### 💵 1번만 구매

In [ ]:
# 1회 구매자용 event_datetime 생성
top_one_df['event_datetime'] = top_one_df['event_date'] + pd.to_timedelta(top_one_df['event_hour'], unit='h')

/tmp/ipython-input-3197304135.py:2: PerformanceWarning: Adding/subtracting object-dtype array to TimedeltaArray not vectorized.
  top_one_df['event_datetime'] = top_one_df['event_date'] + pd.to_timedelta(top_one_df['event_hour'], unit='h')


In [ ]:
# 순서 기반, 유저 기준 전환율 계산 코드
# 순서 기반 전환율 함수 정의
def calculate_ordered_conversion_rate(df):
    # 1. 시간 기준 정렬
    df = df.sort_values(by='event_datetime')

    # 2. 유저별 이벤트 리스트 추출 (시간 순 정렬된 상태)
    user_events = df.groupby('user_id')['event_type'].apply(list)

    # 3. 순차 전환 체크
    view_to_cart = 0
    cart_to_purchase = 0
    view_to_purchase = 0
    total_users = len(user_events)

    for events in user_events:
        if 'view' in events and 'cart' in events:
            if events.index('view') < events.index('cart'):
                view_to_cart += 1
        if 'cart' in events and 'purchase' in events:
            if events.index('cart') < events.index('purchase'):
                cart_to_purchase += 1
        if 'view' in events and 'purchase' in events:
            if events.index('view') < events.index('purchase'):
                view_to_purchase += 1

    # 4. 전환율 계산 (유저 기준)
    view_to_cart_rate = view_to_cart / total_users if total_users else 0
    cart_to_purchase_rate = cart_to_purchase / total_users if total_users else 0
    view_to_purchase_rate = view_to_purchase / total_users if total_users else 0

    return {
        'view_to_cart': view_to_cart_rate,
        'cart_to_purchase': cart_to_purchase_rate,
        'view_to_purchase': view_to_purchase_rate
    }

In [ ]:
# 1회 구매자 전환율 계산
ordered_conv_one = calculate_ordered_conversion_rate(top_one_df)

print("[1회 구매자] 순서 기반 전환율:")
for step, rate in ordered_conv_one.items():
    print(f"{step}: {rate:.2%}")

[1회 구매자] 순서 기반 전환율:
view_to_cart: 80.58%
cart_to_purchase: 60.98%
view_to_purchase: 88.52%


## 💸 2번이상 구매

In [ ]:
# 다회 구매자용 event_datetime 생성
top_multi_df['event_datetime'] = top_multi_df['event_date'] + pd.to_timedelta(top_multi_df['event_hour'], unit='h')

In [ ]:
# 다회 구매자 전환율 계산
ordered_conv_multi = calculate_ordered_conversion_rate(top_multi_df)

print("\n[다회 구매자] 순서 기반 전환율:")
for step, rate in ordered_conv_multi.items():
    print(f"{step}: {rate:.2%}")


[다회 구매자] 순서 기반 전환율:
view_to_cart: 86.73%
cart_to_purchase: 59.50%
view_to_purchase: 88.77%


## 📈 비주력상품 EDA

### 💵 1번만 구매

## 💸 2번이상 구매

### 🔎 유저 수 기반 전환율 <비주력 상품의 다회 구매자 데이터셋 전체> - 기존 방식

In [ ]:
# 다회 구매자용 event_datetime 생성
notop_multi_df['event_datetime'] = notop_multi_df['event_date'] + pd.to_timedelta(notop_multi_df['event_hour'], unit='h')

In [ ]:
# 순서 기반, 유저 기준 전환율 계산 코드
# 순서 기반 전환율 함수 정의
def calculate_ordered_conversion_rate(df):
    # 1. 시간 기준 정렬
    df = df.sort_values(by='event_datetime')

    # 2. 유저별 이벤트 리스트 추출 (시간 순 정렬된 상태)
    user_events = df.groupby('user_id')['event_type'].apply(list)

    # 3. 순차 전환 체크
    view_to_cart = 0
    cart_to_purchase = 0
    view_to_purchase = 0
    total_users = len(user_events)

    for events in user_events:
        if 'view' in events and 'cart' in events:
            if events.index('view') < events.index('cart'):
                view_to_cart += 1
        if 'cart' in events and 'purchase' in events:
            if events.index('cart') < events.index('purchase'):
                cart_to_purchase += 1
        if 'view' in events and 'purchase' in events:
            if events.index('view') < events.index('purchase'):
                view_to_purchase += 1

    # 4. 전환율 계산 (유저 기준)
    view_to_cart_rate = view_to_cart / total_users if total_users else 0
    cart_to_purchase_rate = cart_to_purchase / total_users if total_users else 0
    view_to_purchase_rate = view_to_purchase / total_users if total_users else 0

    return {
        'view_to_cart': view_to_cart_rate,
        'cart_to_purchase': cart_to_purchase_rate,
        'view_to_purchase': view_to_purchase_rate
    }


In [ ]:
# 다회 구매자 전환율 계산
ordered_conv_multi = calculate_ordered_conversion_rate(notop_multi_df)

print("\n[다회 구매자] 순서 기반 전환율:")
for step, rate in ordered_conv_multi.items():
    print(f"{step}: {rate:.2%}")


[다회 구매자] 순서 기반 전환율:
view_to_cart: 89.54%
cart_to_purchase: 62.45%
view_to_purchase: 92.26%


### 🔎 유저 수 기반 전환율<태블로 데이터셋과 동일> => 제갈서현이 디스코드에 보냈던 결과 데이터셋입니다!!!!!!!!!

In [ ]:
# 태블로 대시보드용 샘플링 데이터 df_sampled
# 1) event_date를 datetime으로 변환
df_sampled['event_date'] = pd.to_datetime(df_sampled['event_date'], errors='coerce')

# 2) hour를 Timedelta로 바꿔 더하기
df_sampled['event_datetime'] = df_sampled['event_date'] + pd.to_timedelta(df_sampled['event_hour'], unit='h')


In [ ]:
# 2회 이상 구매자만 필터링
purchase_counts = (
    df_sampled[df_sampled['event_type'] == 'purchase']
    .groupby('user_id')
    .size()
)
multi_buy_users = purchase_counts[purchase_counts >= 2].index

df_multi_buyers = df_sampled[df_sampled['user_id'].isin(multi_buy_users)].copy()


In [ ]:
# 순서 기반, 유저 기준 전환율 계산 코드
# 순서 기반 전환율 함수 정의
def calculate_ordered_conversion_rate(df):
    # 1. 시간 기준 정렬
    df = df.sort_values(by='event_datetime')

    # 2. 유저별 이벤트 리스트 추출 (시간 순 정렬된 상태)
    user_events = df.groupby('user_id')['event_type'].apply(list)

    # 3. 순차 전환 체크
    view_to_cart = 0
    cart_to_purchase = 0
    view_to_purchase = 0
    total_users = len(user_events)

    for events in user_events:
        if 'view' in events and 'cart' in events:
            if events.index('view') < events.index('cart'):
                view_to_cart += 1
        if 'cart' in events and 'purchase' in events:
            if events.index('cart') < events.index('purchase'):
                cart_to_purchase += 1
        if 'view' in events and 'purchase' in events:
            if events.index('view') < events.index('purchase'):
                view_to_purchase += 1

    # 4. 전환율 계산 (유저 기준)
    view_to_cart_rate = view_to_cart / total_users if total_users else 0
    cart_to_purchase_rate = cart_to_purchase / total_users if total_users else 0
    view_to_purchase_rate = view_to_purchase / total_users if total_users else 0

    return {
        'view_to_cart': view_to_cart_rate,
        'cart_to_purchase': cart_to_purchase_rate,
        'view_to_purchase': view_to_purchase_rate
    }

In [ ]:
# 다회 구매자 전환율 계산
ordered_conv_multi = calculate_ordered_conversion_rate(df_multi_buyers)

print("\n[비주력샘플_다회 구매자] 순서 기반 전환율:")
for step, rate in ordered_conv_multi.items():
    print(f"{step}: {rate:.2%}")


[비주력샘플_다회 구매자] 순서 기반 전환율:
view_to_cart: 53.63%
cart_to_purchase: 31.21%
view_to_purchase: 73.46%


원래 전환율에 분모, 분자 집계수 확인 코드 추가

In [ ]:
def calculate_ordered_conversion_rate_verbose_same(df):
    # 원래 코드와 동일: event_datetime만으로 정렬 (추가 키 없음)
    df_sorted = df.sort_values(by='event_datetime')

    # 유저별 이벤트 리스트 (원래와 동일)
    user_events = df_sorted.groupby('user_id')['event_type'].apply(list)

    total_users = len(user_events)  # 분모(세 단계 공통)

    # 원래 방식 그대로: 각 유저당 최대 1회 카운트
    v2c_num = 0
    c2p_num = 0
    v2p_num = 0

    # (원하면 어떤 유저가 분자에 들어갔는지 확인할 수 있도록 집합도 유지)
    v2c_users, c2p_users, v2p_users = set(), set(), set()

    for uid, events in user_events.items():
        # view -> cart
        if 'view' in events and 'cart' in events:
            if events.index('view') < events.index('cart'):
                v2c_num += 1
                v2c_users.add(uid)

        # cart -> purchase
        if 'cart' in events and 'purchase' in events:
            if events.index('cart') < events.index('purchase'):
                c2p_num += 1
                c2p_users.add(uid)

        # view -> purchase
        if 'view' in events and 'purchase' in events:
            if events.index('view') < events.index('purchase'):
                v2p_num += 1
                v2p_users.add(uid)

    # 분모(세 단계 동일)
    den = total_users

    return {
        'view_to_cart': {
            'numerator': v2c_num,
            'denominator': den,
            'not_converted': den - v2c_num,
            'rate': (v2c_num / den) if den else 0.0,
            'user_ids': v2c_users  # 필요 없으면 지워도 돼요
        },
        'cart_to_purchase': {
            'numerator': c2p_num,
            'denominator': den,
            'not_converted': den - c2p_num,
            'rate': (c2p_num / den) if den else 0.0,
            'user_ids': c2p_users
        },
        'view_to_purchase': {
            'numerator': v2p_num,
            'denominator': den,
            'not_converted': den - v2p_num,
            'rate': (v2p_num / den) if den else 0.0,
            'user_ids': v2p_users
        },
        'total_users': den
    }

# 사용 예시 (df_multi_buyers 네가 쓰던 동일 데이터프레임)
res_same = calculate_ordered_conversion_rate_verbose_same(df_multi_buyers)
for k, v in res_same.items():
    if k == 'total_users':
        continue
    print(f"{k}: 분자={v['numerator']:,} / 분모={v['denominator']:,} "
          f"(미전환={v['not_converted']:,}) → 전환율={v['rate']:.2%}")
print(f"총 유저 수(total_users): {res_same['total_users']:,}")

view_to_cart: 분자=4,825 / 분모=8,997 (미전환=4,172) → 전환율=53.63%
cart_to_purchase: 분자=2,808 / 분모=8,997 (미전환=6,189) → 전환율=31.21%
view_to_purchase: 분자=6,609 / 분모=8,997 (미전환=2,388) → 전환율=73.46%
총 유저 수(total_users): 8,997


### 🔎 유저 수 기반 전환율<태블로 데이터셋과 동일+태블로 산정 방식과 최대한 맞춰서 돌려본 코드> - 지연님 결과와 비교 위해

In [ ]:
# 전처리: event_datetime 생성 (이미 있으면 생략)
df = df_sampled.copy()
df['event_date'] = pd.to_datetime(df['event_date'], errors='coerce')
df['event_datetime'] = df['event_date'] + pd.to_timedelta(df['event_hour'], unit='h')
df = df.dropna(subset=['event_datetime'])

# 2회 이상 구매자만 (태블로와 동일한 '기간/필터'로 산출했는지 꼭 확인!)
purchase_counts = (
    df[df['event_type'] == 'purchase']
    .groupby('user_id')
    .size()
)
multi_users = purchase_counts[purchase_counts >= 2].index
df = df[df['user_id'].isin(multi_users)].copy()

# (권장) 같은 시각 타이에 대해 안정 정렬: view < cart < purchase 우선순위 부여
# 1) 이벤트 우선순위 정의 (동시간대 타이 깨기)
order = {'view': 0, 'cart': 1, 'purchase': 2}

# 2) 우선순위를 숫자로 붙이기
df['etype_order'] = df['event_type'].map(order)

# 3) 정렬: 같은 유저·같은 시각이면 view → cart → purchase 순서로
df = df.sort_values(['user_id', 'event_datetime', 'etype_order'])


# 각 유저의 '첫 발생 시각' 피벗 (태블로에서 첫 이벤트 비교와 동일)
first_ts = (
    df.groupby(['user_id', 'event_type'])['event_datetime']
      .min()
      .unstack()  # columns: view, cart, purchase (있을 때만 생성)
)

has_view     = first_ts.get('view').notna()     if 'view' in first_ts else first_ts.index.to_series().eq(False)
has_cart     = first_ts.get('cart').notna()     if 'cart' in first_ts else first_ts.index.to_series().eq(False)
has_purchase = first_ts.get('purchase').notna() if 'purchase' in first_ts else first_ts.index.to_series().eq(False)

# 전환 판정 (동시간은 미전환 처리하려면 '<' 사용, 동시간도 전환으로 치려면 '<=' 로 바꾸세요)
v2c_conv = has_view & has_cart & (first_ts['view'] <= first_ts['cart'])
c2p_conv = has_cart & has_purchase & (first_ts['cart'] <= first_ts['purchase'])
v2p_conv = has_view & has_purchase & (first_ts['view'] <= first_ts['purchase'])

# 태블로의 TOTAL(COUNTD([User Id]))에 해당하는 '단계별 분모'
den_v2c = has_view.sum()     # View → Cart 단계의 전체 유저(=해당 도넛의 TOTAL)
den_c2p = has_cart.sum()     # Cart → Purchase 단계의 전체 유저
den_v2p = has_view.sum()     # View → Purchase 단계의 전체 유저

# COUNTD([User Id])에 해당하는 '전환 유저 수(분자)'
num_v2c = v2c_conv.sum()
num_c2p = c2p_conv.sum()
num_v2p = v2p_conv.sum()

rate_v2c = num_v2c / den_v2c if den_v2c else 0.0
rate_c2p = num_c2p / den_c2p if den_c2p else 0.0
rate_v2p = num_v2p / den_v2p if den_v2p else 0.0

print({
    'view→cart': {'분자': int(num_v2c), '분모': int(den_v2c), '전환율': rate_v2c},
    'cart→purchase': {'분자': int(num_c2p), '분모': int(den_c2p), '전환율': rate_c2p},
    'view→purchase': {'분자': int(num_v2p), '분모': int(den_v2p), '전환율': rate_v2p},
})

# (도넛처럼 전환/미전환 분할이 필요하면:)
v2c_not = den_v2c - num_v2c
c2p_not = den_c2p - num_c2p
v2p_not = den_v2p - num_v2p


{'view→cart': {'분자': 5087, '분모': 8484, '전환율': np.float64(0.5995992456388496)}, 'cart→purchase': {'분자': 3347, '분모': 6062, '전환율': np.float64(0.5521280105575718)}, 'view→purchase': {'분자': 7096, '분모': 8484, '전환율': np.float64(0.8363979255068364)}}


In [ ]:
print(v2c_not)
print(c2p_not)
print(v2p_not)

3397
2715
1388
